In [129]:
import numpy as np
import pandas as pd
import sys
import re
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
model_path = '/Users/lambert/Documents/Python_code/tensorflow_code/Titanic/Titanic_model/'

In [130]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [131]:
import re
def get_title(name):
    if pd.isnull(name):
        return 'Null'
    title_search = re.search('([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1).lower()
    else:
        return 'None'

In [132]:
titles = {'mr':1, 'mrs':2, 'mme':2, 'ms':3, 'miss':3, 'mlle':3, 'don':4, 'sir':4, 'jonkheer':4,
          'major':4, 'col':4, 'dr':4, 'master':4, 'capt':4, 'dona':5, 'lady':5, 'countess':5,
         'rev':6}
df_train['Title'] = df_train['Name'].apply(lambda name: titles.get(get_title(name)))
df_train['Honor'] = df_train['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)
df_test['Title'] = df_test['Name'].apply(lambda name: titles.get(get_title(name)))
df_test['Honor'] = df_test['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)

In [133]:
# 1. Data clearning
# df_train.Age.fillna(fill_df_Age, inplace=True)
# df_test.Age.fillna(fill_df_Age, inplace=True)
for i in range(1,7):
    a = df_train[(df_train.Age.isnull()) & (df_train.Title==i)]
    df_train.Age.iloc[a.index] = df_train.Age[df_train.Title==i].median()
    b = df_test[(df_test.Age.isnull()) & (df_test.Title==i)]
    df_test.Age.iloc[b.index] = df_train.Age[df_train.Title==i].median()

# Cabin - fill nan with 'fill' and select first cabin
df_train.Cabin.fillna('fill',inplace=True)
df_train.Cabin = df_train.Cabin.map(lambda x:x.split(' ')[0])
df_test.Cabin.fillna('fill', inplace=True)
df_test.Cabin = df_test.Cabin.map(lambda x:x.split(' ')[0])

# Sex - male: 1, female: 1
df_train.Sex.replace({'male':1, 'female':0}, inplace=True)
df_test.Sex.replace({'male':1, 'female':0}, inplace=True)

df_train = df_train.dropna(axis=0, how='any')
df_test.Fare.fillna(df_test.Fare.mean(), inplace=True)

# Create a 'Deceased' column for sconed class
df_train['Deceased'] = df_train.Survived.apply(lambda s: int(not s))

/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [134]:
# 2. Select the features
sFeatures = ['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',             
 'Cabin',             
 'Embarked',
 'Title',
 'Honor']

X = df_train[sFeatures]
y = df_train[['Survived','Deceased']]
y = y.values.reshape(len(y),2)
X_test = df_test[sFeatures]

In [135]:
# 3. Label Encoder
# X_test has elements that X doesn't have
# create a Cabin_labels cover X and X_test
X_Cabin_unique = X['Cabin'].unique()
X_lack = X_test.Cabin[X_test['Cabin'].isin(X_Cabin_unique)==0].values
Cabin_labels = np.append(X_Cabin_unique, X_lack)
# print(X_Cabin_unique.shape, X_Cabin_unique)
# print(Cabin_labels.shape, Cabin_labels)

le_Cabin = LabelEncoder().fit(Cabin_labels)
le_Embarked = LabelEncoder().fit(np.array(X['Embarked'].tolist()))

X.loc[:,'Cabin'] = le_Cabin.transform(X['Cabin'])
X.loc[:,'Embarked'] = le_Embarked.transform(X['Embarked'])
X_test.loc[:,'Cabin'] = le_Cabin.transform(X_test['Cabin'])
X_test.loc[:,'Embarked'] = le_Embarked.transform(X_test['Embarked'])
X = X.astype(np.float32)
y = y.astype(np.float32)

/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [136]:
# Validation
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [137]:
X_val.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title,Honor
457,1.0,0.0,35.0,1.0,0.0,51.862499,123.0,2.0,2.0,0.0
661,3.0,1.0,40.0,0.0,0.0,7.225000,180.0,0.0,1.0,0.0
259,2.0,0.0,50.0,0.0,1.0,26.000000,180.0,2.0,2.0,0.0
638,3.0,0.0,41.0,0.0,5.0,39.687500,180.0,2.0,2.0,0.0
742,1.0,0.0,21.0,2.0,2.0,262.375000,46.0,0.0,3.0,0.0


### Tensor Graph

In [138]:
def next_batch(batch, X, y):
    num = np.arange(len(X))
    np.random.shuffle(num)
    select = num[0:batch]
    x_batch = X.iloc[select]
    y_batch = y[select]
    
    return(x_batch, y_batch)

In [139]:
batch_size = 200

def optimize(num_iterations):
    total_loss = 0
    for i in range(num_iterations):
        x_batch, y_batch = next_batch(batch_size, X_train_data, y_train_data)
        feed_dict_train = {X_place: x_batch, y_place: y_batch}
        _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
        total_loss += loss
        print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
    print ('Training complete!')

In [141]:
# Tensorflow variables
features= len(sFeatures)
X_place = tf.placeholder(tf.float32, shape=[None,features])
y_place = tf.placeholder(tf.float32, shape=[None,2])
W = tf.Variable(tf.random_normal([features, 2]), name='weights')
b = tf.Variable(tf.zeros(2), name='bias')
logits = tf.matmul(X_place, W) + b
y_pred = tf.nn.softmax(logits)

In [142]:
# Cross Entropy
learning_rate = 0.001
cross_entropy = -tf.reduce_sum(y_place * tf.log(y_pred + 1e-10), axis=1)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_train)
cost = tf.reduce_mean(cross_entropy)
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
saver = tf.train.Saver()

In [144]:
cost

<tf.Tensor 'Mean_3:0' shape=() dtype=float32>

In [146]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(cost))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_8' with dtype float and shape [?,10]
	 [[Node: Placeholder_8 = Placeholder[dtype=DT_FLOAT, shape=[?,10], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_8', defined at:
  File "/Users/lambert/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/lambert/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/Users/lambert/virtualenvs/tenso

In [116]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

In [117]:
# # training starting option 1
# for epoch in range(10000):
#     total_loss = 0.
#     x_batch, y_batch = next_batch(batch_size, X_train, y_train)
#     feed_dict_train = {X_place: x_batch, y_place: y_batch}
#     _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
#     total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [118]:
# sess.close()

In [120]:
# training starting option 1 with CPU
# for epoch in range(100):
#     total_loss = 0.
#     for i in range(len(X_train)):
#         feed = {X_place: X_train, y_place: y_train}
#         _, loss = sess.run([train_op, cost], feed_dict=feed)
#         total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [122]:
# training starting option 1 with GPU
# with tf.device('/device:GPU:0'):
start = time.time()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(10000):
        total_loss = 0.
        for i in range(len(X_train)):
            feed = {X_place: X_train, y_place: y_train}
            _, loss = sess.run([train_op, cost], feed_dict=feed)
            total_loss += loss
        print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
    #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
    #     print('predict: {:.6f}'.format(y_pred))
    print ('Training complete!')

    feed_dict_train = {X_place: X_train}
    prediction = sess.run(y_pred, feed_dict=feed_dict_train)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train,axis=1))
    # evaluate the accuracy
    accuracy = np.mean(correct)
    print('Accuracy on validation: {:2.2%}'.format (accuracy))

end = time.time()
print('time spend: {:.4f} sec'.format(end-start))

Epoch: 0001, total loss=5894.619804382
Epoch: 0002, total loss=5894.619804382
Epoch: 0003, total loss=5894.619804382
Epoch: 0004, total loss=5894.619804382
Epoch: 0005, total loss=5894.619804382
Epoch: 0006, total loss=5894.619804382
Epoch: 0007, total loss=5894.619804382
Epoch: 0008, total loss=5894.619804382
Epoch: 0009, total loss=5894.619804382
Epoch: 0010, total loss=5894.619804382
Epoch: 0011, total loss=5894.619804382
Epoch: 0012, total loss=5894.619804382
Epoch: 0013, total loss=5894.619804382
Epoch: 0014, total loss=5894.619804382
Epoch: 0015, total loss=5894.619804382
Epoch: 0016, total loss=5894.619804382
Epoch: 0017, total loss=5894.619804382
Epoch: 0018, total loss=5894.619804382
Epoch: 0019, total loss=5894.619804382
Epoch: 0020, total loss=5894.619804382
Epoch: 0021, total loss=5894.619804382
Epoch: 0022, total loss=5894.619804382
Epoch: 0023, total loss=5894.619804382
Epoch: 0024, total loss=5894.619804382
Epoch: 0025, total loss=5894.619804382
Epoch: 0026, total loss=5

Epoch: 0212, total loss=5894.619804382
Epoch: 0213, total loss=5894.619804382
Epoch: 0214, total loss=5894.619804382
Epoch: 0215, total loss=5894.619804382
Epoch: 0216, total loss=5894.619804382
Epoch: 0217, total loss=5894.619804382
Epoch: 0218, total loss=5894.619804382
Epoch: 0219, total loss=5894.619804382
Epoch: 0220, total loss=5894.619804382
Epoch: 0221, total loss=5894.619804382
Epoch: 0222, total loss=5894.619804382
Epoch: 0223, total loss=5894.619804382
Epoch: 0224, total loss=5894.619804382
Epoch: 0225, total loss=5894.619804382
Epoch: 0226, total loss=5894.619804382
Epoch: 0227, total loss=5894.619804382
Epoch: 0228, total loss=5894.619804382
Epoch: 0229, total loss=5894.619804382
Epoch: 0230, total loss=5894.619804382
Epoch: 0231, total loss=5894.619804382
Epoch: 0232, total loss=5894.619804382
Epoch: 0233, total loss=5894.619804382
Epoch: 0234, total loss=5894.619804382
Epoch: 0235, total loss=5894.619804382
Epoch: 0236, total loss=5894.619804382
Epoch: 0237, total loss=5

Epoch: 0423, total loss=5894.619804382
Epoch: 0424, total loss=5894.619804382
Epoch: 0425, total loss=5894.619804382
Epoch: 0426, total loss=5894.619804382
Epoch: 0427, total loss=5894.619804382
Epoch: 0428, total loss=5894.619804382
Epoch: 0429, total loss=5894.619804382
Epoch: 0430, total loss=5894.619804382
Epoch: 0431, total loss=5894.619804382
Epoch: 0432, total loss=5894.619804382
Epoch: 0433, total loss=5894.619804382
Epoch: 0434, total loss=5894.619804382
Epoch: 0435, total loss=5894.619804382
Epoch: 0436, total loss=5894.619804382
Epoch: 0437, total loss=5894.619804382
Epoch: 0438, total loss=5894.619804382
Epoch: 0439, total loss=5894.619804382
Epoch: 0440, total loss=5894.619804382
Epoch: 0441, total loss=5894.619804382
Epoch: 0442, total loss=5894.619804382
Epoch: 0443, total loss=5894.619804382
Epoch: 0444, total loss=5894.619804382
Epoch: 0445, total loss=5894.619804382
Epoch: 0446, total loss=5894.619804382
Epoch: 0447, total loss=5894.619804382
Epoch: 0448, total loss=5

Epoch: 0634, total loss=5894.619804382
Epoch: 0635, total loss=5894.619804382
Epoch: 0636, total loss=5894.619804382
Epoch: 0637, total loss=5894.619804382
Epoch: 0638, total loss=5894.619804382
Epoch: 0639, total loss=5894.619804382
Epoch: 0640, total loss=5894.619804382
Epoch: 0641, total loss=5894.619804382
Epoch: 0642, total loss=5894.619804382
Epoch: 0643, total loss=5894.619804382
Epoch: 0644, total loss=5894.619804382
Epoch: 0645, total loss=5894.619804382
Epoch: 0646, total loss=5894.619804382
Epoch: 0647, total loss=5894.619804382
Epoch: 0648, total loss=5894.619804382
Epoch: 0649, total loss=5894.619804382
Epoch: 0650, total loss=5894.619804382
Epoch: 0651, total loss=5894.619804382
Epoch: 0652, total loss=5894.619804382
Epoch: 0653, total loss=5894.619804382
Epoch: 0654, total loss=5894.619804382
Epoch: 0655, total loss=5894.619804382
Epoch: 0656, total loss=5894.619804382
Epoch: 0657, total loss=5894.619804382
Epoch: 0658, total loss=5894.619804382
Epoch: 0659, total loss=5

Epoch: 0845, total loss=5894.619804382
Epoch: 0846, total loss=5894.619804382
Epoch: 0847, total loss=5894.619804382
Epoch: 0848, total loss=5894.619804382
Epoch: 0849, total loss=5894.619804382
Epoch: 0850, total loss=5894.619804382
Epoch: 0851, total loss=5894.619804382
Epoch: 0852, total loss=5894.619804382
Epoch: 0853, total loss=5894.619804382
Epoch: 0854, total loss=5894.619804382
Epoch: 0855, total loss=5894.619804382
Epoch: 0856, total loss=5894.619804382
Epoch: 0857, total loss=5894.619804382
Epoch: 0858, total loss=5894.619804382
Epoch: 0859, total loss=5894.619804382
Epoch: 0860, total loss=5894.619804382
Epoch: 0861, total loss=5894.619804382
Epoch: 0862, total loss=5894.619804382
Epoch: 0863, total loss=5894.619804382
Epoch: 0864, total loss=5894.619804382
Epoch: 0865, total loss=5894.619804382
Epoch: 0866, total loss=5894.619804382
Epoch: 0867, total loss=5894.619804382
Epoch: 0868, total loss=5894.619804382
Epoch: 0869, total loss=5894.619804382
Epoch: 0870, total loss=5

...

Epoch: 9074, total loss=5894.619804382
Epoch: 9075, total loss=5894.619804382
Epoch: 9076, total loss=5894.619804382
Epoch: 9077, total loss=5894.619804382
Epoch: 9078, total loss=5894.619804382
Epoch: 9079, total loss=5894.619804382
Epoch: 9080, total loss=5894.619804382
Epoch: 9081, total loss=5894.619804382
Epoch: 9082, total loss=5894.619804382
Epoch: 9083, total loss=5894.619804382
Epoch: 9084, total loss=5894.619804382
Epoch: 9085, total loss=5894.619804382
Epoch: 9086, total loss=5894.619804382
Epoch: 9087, total loss=5894.619804382
Epoch: 9088, total loss=5894.619804382
Epoch: 9089, total loss=5894.619804382
Epoch: 9090, total loss=5894.619804382
Epoch: 9091, total loss=5894.619804382
Epoch: 9092, total loss=5894.619804382
Epoch: 9093, total loss=5894.619804382
Epoch: 9094, total loss=5894.619804382
Epoch: 9095, total loss=5894.619804382
Epoch: 9096, total loss=5894.619804382
Epoch: 9097, total loss=5894.619804382
Epoch: 9098, total loss=5894.619804382
Epoch: 9099, total loss=5

Epoch: 9285, total loss=5894.619804382
Epoch: 9286, total loss=5894.619804382
Epoch: 9287, total loss=5894.619804382
Epoch: 9288, total loss=5894.619804382
Epoch: 9289, total loss=5894.619804382
Epoch: 9290, total loss=5894.619804382
Epoch: 9291, total loss=5894.619804382
Epoch: 9292, total loss=5894.619804382
Epoch: 9293, total loss=5894.619804382
Epoch: 9294, total loss=5894.619804382
Epoch: 9295, total loss=5894.619804382
Epoch: 9296, total loss=5894.619804382
Epoch: 9297, total loss=5894.619804382
Epoch: 9298, total loss=5894.619804382
Epoch: 9299, total loss=5894.619804382
Epoch: 9300, total loss=5894.619804382
Epoch: 9301, total loss=5894.619804382
Epoch: 9302, total loss=5894.619804382
Epoch: 9303, total loss=5894.619804382
Epoch: 9304, total loss=5894.619804382
Epoch: 9305, total loss=5894.619804382
Epoch: 9306, total loss=5894.619804382
Epoch: 9307, total loss=5894.619804382
Epoch: 9308, total loss=5894.619804382
Epoch: 9309, total loss=5894.619804382
Epoch: 9310, total loss=5

Epoch: 9496, total loss=5894.619804382
Epoch: 9497, total loss=5894.619804382
Epoch: 9498, total loss=5894.619804382
Epoch: 9499, total loss=5894.619804382
Epoch: 9500, total loss=5894.619804382
Epoch: 9501, total loss=5894.619804382
Epoch: 9502, total loss=5894.619804382
Epoch: 9503, total loss=5894.619804382
Epoch: 9504, total loss=5894.619804382
Epoch: 9505, total loss=5894.619804382
Epoch: 9506, total loss=5894.619804382
Epoch: 9507, total loss=5894.619804382
Epoch: 9508, total loss=5894.619804382
Epoch: 9509, total loss=5894.619804382
Epoch: 9510, total loss=5894.619804382
Epoch: 9511, total loss=5894.619804382
Epoch: 9512, total loss=5894.619804382
Epoch: 9513, total loss=5894.619804382
Epoch: 9514, total loss=5894.619804382
Epoch: 9515, total loss=5894.619804382
Epoch: 9516, total loss=5894.619804382
Epoch: 9517, total loss=5894.619804382
Epoch: 9518, total loss=5894.619804382
Epoch: 9519, total loss=5894.619804382
Epoch: 9520, total loss=5894.619804382
Epoch: 9521, total loss=5

Epoch: 9707, total loss=5894.619804382
Epoch: 9708, total loss=5894.619804382
Epoch: 9709, total loss=5894.619804382
Epoch: 9710, total loss=5894.619804382
Epoch: 9711, total loss=5894.619804382
Epoch: 9712, total loss=5894.619804382
Epoch: 9713, total loss=5894.619804382
Epoch: 9714, total loss=5894.619804382
Epoch: 9715, total loss=5894.619804382
Epoch: 9716, total loss=5894.619804382
Epoch: 9717, total loss=5894.619804382
Epoch: 9718, total loss=5894.619804382
Epoch: 9719, total loss=5894.619804382
Epoch: 9720, total loss=5894.619804382
Epoch: 9721, total loss=5894.619804382
Epoch: 9722, total loss=5894.619804382
Epoch: 9723, total loss=5894.619804382
Epoch: 9724, total loss=5894.619804382
Epoch: 9725, total loss=5894.619804382
Epoch: 9726, total loss=5894.619804382
Epoch: 9727, total loss=5894.619804382
Epoch: 9728, total loss=5894.619804382
Epoch: 9729, total loss=5894.619804382
Epoch: 9730, total loss=5894.619804382
Epoch: 9731, total loss=5894.619804382
Epoch: 9732, total loss=5

Epoch: 9918, total loss=5894.619804382
Epoch: 9919, total loss=5894.619804382
Epoch: 9920, total loss=5894.619804382
Epoch: 9921, total loss=5894.619804382
Epoch: 9922, total loss=5894.619804382
Epoch: 9923, total loss=5894.619804382
Epoch: 9924, total loss=5894.619804382
Epoch: 9925, total loss=5894.619804382
Epoch: 9926, total loss=5894.619804382
Epoch: 9927, total loss=5894.619804382
Epoch: 9928, total loss=5894.619804382
Epoch: 9929, total loss=5894.619804382
Epoch: 9930, total loss=5894.619804382
Epoch: 9931, total loss=5894.619804382
Epoch: 9932, total loss=5894.619804382
Epoch: 9933, total loss=5894.619804382
Epoch: 9934, total loss=5894.619804382
Epoch: 9935, total loss=5894.619804382
Epoch: 9936, total loss=5894.619804382
Epoch: 9937, total loss=5894.619804382
Epoch: 9938, total loss=5894.619804382
Epoch: 9939, total loss=5894.619804382
Epoch: 9940, total loss=5894.619804382
Epoch: 9941, total loss=5894.619804382
Epoch: 9942, total loss=5894.619804382
Epoch: 9943, total loss=5

In [206]:
save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
# sess.close()

In [207]:
# evaluate the training data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_train = {X_place: X_train_data}
    prediction = sess.run(y_pred, feed_dict=feed_dict_train)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train_data,axis=1))
# evaluate the accuracy
accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

INFO:tensorflow:Restoring parameters from /Users/lambert/Documents/Python_code/tensorflow_code/Titanic/Titanic_model/Titanic_model.ckpt
Accuracy on validation: 63.06%


In [210]:
# evaluate the validate data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_valid = {X_place: X_val_data}
    prediction = sess.run(y_pred, feed_dict=feed_dict_valid)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_val_data,axis=1))
# evaluate the accuracy

accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

INFO:tensorflow:Restoring parameters from /Users/lambert/Documents/Python_code/tensorflow_code/Titanic/Titanic_model/Titanic_model.ckpt
Accuracy on validation: 64.13%


In [224]:
# prediction for test data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_test = {X_place: X_test}
    prediction = sess.run(y_pred, feed_dict=feed_dict_test)
    y_predict = np.argmax(prediction, axis=1)
# evaluate the accuracy

INFO:tensorflow:Restoring parameters from /Users/lambert/Documents/Python_code/tensorflow_code/Titanic/Titanic_model/Titanic_model.ckpt


In [223]:
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_random.csv')

407

In [226]:
y_predict.shape

(418,)

In [220]:
# print_accuracy()
# plot_example_errors()
# optimize(num_iterations=10)
print_accuracy()
# plot_example_errors()
# plot_weights()

NameError: name 'print_accuracy' is not defined

In [ ]:
File_Writer=tf.summary.FileWriter('/Users/lambert/Documents/Python_code/tensorflow_code/Titanic'
                                  ,sess.graph)

In [ ]:
# with tf.device('/device:GPU:0'):
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:

In [ ]:
# allocate as much GPU memory based on runtime allocations
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
# allocate 40% memory of GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config, ...)

In [ ]:
# Automatically choose existing in case specified one doesn't exit
# allow_soft_placement=True
with tf.device('/device:GPU:2'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with allow_soft_placement and log_device_placement set
# to True.
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True))

In [76]:
sys.argv[2]

'/Users/lambert/Library/Jupyter/runtime/kernel-d05fde7f-2b3b-40a0-9219-fbcb703d8987.json'

In [69]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2731858689698339022]

Logistic Regression

In [ ]:
# Logistic Regression
lr = LogisticRegression(C=1, max_iter=100, solver='lbfgs').fit(X_train, y_train)
yp = lr.predict(X_train)
y_predict = lr.predict(X_test)

passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_knn.csv')
# passengerid = np.array(df_train.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': yp}).set_index('PassengerId')
# submission.to_csv('y_train_predict.csv')

Gradient Boosting Classifier

In [ ]:
# Gradient Boosting Classifier
# gbdt = GradientBoostingClassifier(n_estimators=300, max_depth=5).fit(X_train, y_train)
# y_predict = gbdt.predict(X_test)

# passengerid = np.array(df_test.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
# submission.to_csv('submission.csv')

Random forests

In [ ]:
# RandomForest
rfc = RandomForestClassifier(n_estimators=200, max_depth=3).fit(X_train, y_train)
y_predict = rfc.predict(X_test)
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_random.csv')

Stochastic Gradient Descent (SDG)

In [ ]:
# Stochastic Gradient Descent (SGD) easily scale with more than 10^5 samples and features
sgd = SGDClassifier(alpha=0.01, max_iter=1000).fit(X_train, y_train)
y_predict = sgd.predict(X_test)
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_sdg.csv')